In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('Telco-Customer-Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [13]:
df.info

<bound method DataFrame.info of       customerID  gender  SeniorCitizen Partner Dependents  tenure  \
0     7590-VHVEG  Female              0     Yes         No       1   
1     5575-GNVDE    Male              0      No         No      34   
2     3668-QPYBK    Male              0      No         No       2   
3     7795-CFOCW    Male              0      No         No      45   
4     9237-HQITU  Female              0      No         No       2   
...          ...     ...            ...     ...        ...     ...   
7038  6840-RESVB    Male              0     Yes        Yes      24   
7039  2234-XADUH  Female              0     Yes        Yes      72   
7040  4801-JZAZL  Female              0     Yes        Yes      11   
7041  8361-LTMKD    Male              1     Yes         No       4   
7042  3186-AJIEK    Male              0      No         No      66   

     PhoneService     MultipleLines InternetService OnlineSecurity  ...  \
0              No  No phone service             DSL 

In [15]:
pd.isnull(df).sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [16]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
# filling TotalCharges NaN with (MonthlyCharges * tenure) as reasonable proxy
mask = df['TotalCharges'].isna()
if mask.any():
    df.loc[mask, 'TotalCharges'] = (df.loc[mask, 'MonthlyCharges'] * df.loc[mask, 'tenure']).round(2)

# creating churn flag numeric
if 'Churn' in df.columns:
    df['Churn_flag'] = df['Churn'].map({'Yes':1, 'No':0})
else:
    raise KeyError("Column 'Churn' not found in dataset")

In [20]:
# seed for reproducibility
rng = np.random.default_rng(42)

# 1) Customer_Lifetime_Value (CLV) — proxy: MonthlyCharges * expected_remaining_months
contract_map = {'Month-to-month': 6, 'One year': 18, 'Two year': 36}

# Handling variants
df['Contract'] = df['Contract'].astype(str).str.strip()
df['expected_remaining_months'] = df['Contract'].map(contract_map).fillna(6) + rng.integers(-3, 9, size=len(df))
df['expected_remaining_months'] = df['expected_remaining_months'].clip(lower=0)
df['Customer_Lifetime_Value'] = (df['MonthlyCharges'] * df['expected_remaining_months']).round(2)

# 2) Discount_Rate (%) 
base = rng.normal(loc=2.0, scale=1.0, size=len(df))  # around 2%
contract_bonus = df['Contract'].map({'Month-to-month':0.0, 'One year':1.0, 'Two year':2.0}).fillna(0.0)
tenure_bonus = (df['tenure'] / (df['tenure'].max() + 1)) * 2.0  # up to +2%
df['Discount_Rate'] = (base + contract_bonus + tenure_bonus).clip(lower=0).round(2)  # percent

# 3) Support_Tickets_Opened — Poisson Distribution around service usage
service_cols = ['PhoneService','InternetService','OnlineSecurity','OnlineBackup',
                'DeviceProtection','TechSupport','StreamingTV','StreamingMovies']
existing = [c for c in service_cols if c in df.columns]
# mapping "No"/"Yes" and other strings to 0/1 for counting usage
usage_df = pd.DataFrame()
for c in existing:
    usage_df[c] = np.where(df[c].astype(str).str.strip().str.lower() == 'no', 0, 1)
df['service_count'] = usage_df.sum(axis=1)

# lambda for Poisson: baseline 0.3 + 0.2*service_count
lam += np.where(df['Contract']=='Month-to-month', 0.1, 0)

# Generating Poisson-distributed ticket counts
tickets = rng.poisson(lam=lam)
df['Support_Tickets_Opened'] = np.clip(tickets, 0, None).astype(int)


# 4) Engagement_Score (0-100) — higher when service_count high and fewer support tickets
svc_norm = (df['service_count'] / (df['service_count'].max() + 1)) * 0.75
tickets_penalty = (df['Support_Tickets_Opened'] / (df['Support_Tickets_Opened'].max() + 1)) * 0.5
rand_noise = rng.normal(0, 0.05, size=len(df))
eng_raw = 0.25 + svc_norm - tickets_penalty + rand_noise
df['Engagement_Score'] = (eng_raw.clip(0,1) * 100).round(1)

df.drop(columns=['expected_remaining_months','service_count'], inplace=True, errors='ignore')

cols = list(df.columns)
for col in ['Customer_Lifetime_Value','Discount_Rate','Engagement_Score','Support_Tickets_Opened','Churn_flag']:
    if col in cols:
        cols.insert(cols.index('MonthlyCharges')+1, cols.pop(cols.index(col)))
df = df[cols]

In [25]:
df.to_csv('telecom-Hybrid.csv', index=False)

In [26]:
print("Rows:", df.shape[0], "Cols:", df.shape[1])
print("Sample new columns:")
print(df[['Customer_Lifetime_Value','Discount_Rate','Engagement_Score','Support_Tickets_Opened','Churn_flag']].head().to_dict(orient='records'))

Rows: 7043 Cols: 26
Sample new columns:
[{'Customer_Lifetime_Value': 119.4, 'Discount_Rate': 4.01, 'Engagement_Score': 45.8, 'Support_Tickets_Opened': 0, 'Churn_flag': 0}, {'Customer_Lifetime_Value': 1366.8, 'Discount_Rate': 3.98, 'Engagement_Score': 51.4, 'Support_Tickets_Opened': 2, 'Churn_flag': 0}, {'Customer_Lifetime_Value': 538.5, 'Discount_Rate': 2.12, 'Engagement_Score': 57.2, 'Support_Tickets_Opened': 1, 'Churn_flag': 1}, {'Customer_Lifetime_Value': 846.0, 'Discount_Rate': 3.86, 'Engagement_Score': 39.1, 'Support_Tickets_Opened': 2, 'Churn_flag': 0}, {'Customer_Lifetime_Value': 565.6, 'Discount_Rate': 3.16, 'Engagement_Score': 26.2, 'Support_Tickets_Opened': 2, 'Churn_flag': 1}]
